In [29]:
puzzle = open('puzzle.txt', 'r').read()
example = open('example.txt', 'r').read()

In [30]:
input = puzzle

schematics = input.split('\n')

# Part 1

In [31]:
# Parse input

import re
target_lights_list, buttons_list, joltages_list = [],[],[]

for s in schematics:
    target_lights, buttons, joltages = map(str.strip,re.search(r'\[(.*)\](.*)\{(.*)\}',s).groups())

    initial_lights = [0 for l in target_lights]
    target_lights = [True if l == '#' else False for l in target_lights]
    buttons = [eval(b.replace('(','[').replace(')',']')) for b in buttons.split(' ')]
    joltages = [int(j) for j in joltages.split(',')]

    target_lights_list.append(target_lights)
    buttons_list.append(buttons)
    joltages_list.append(joltages)

In [32]:
# Each button only needs to be pressed a maximum of 1 time (since it only flips the same lights on or off i.e. is binary)

# Can iterate through all combinations of buttons to press, check the output and then if valid return button press

import itertools

def update_lights(lights,button):
    for idx in button:
        lights[idx] = not lights[idx]
    return lights

def find_max_button_presses(target_lights,buttons):
    num_buttons = len(buttons)
    for num_buttons_pressed in range(1,num_buttons+1):
        for buttons_pressed in itertools.combinations(buttons,num_buttons_pressed):
            cur_lights = [False for _ in target_lights]
            for button in buttons_pressed:
                cur_lights = update_lights(cur_lights,button)
            if cur_lights == target_lights:
                return num_buttons_pressed

N = len(schematics)


count = 0
for i in range(N):
    count += find_max_button_presses(target_lights_list[i],buttons_list[i])

count

452

# Part 2

In [33]:
from z3 import *

total_button_pushes = 0

N = len(schematics)

# Iterate through schematics
for i in range(N):
    # Extract current schematic
    buttons, joltages = buttons_list[i], joltages_list[i]

    # Set up solver
    opt_solver = Optimize()

    num_buttons = len(buttons)

    # Set up variables for creating equations in the optimisation solver
    opt_solver_vars = []
    for i in range(num_buttons):
        opt_solver_vars.append(Int(f'x{i}'))
    
    # Force >= 0 button presses
    for var in opt_solver_vars:
        opt_solver.add(var >= 0)

    # Set up equations that describe relationship between buttons and joltage
    for joltage_idx in range(len(joltages)):
        target_joltage = joltages[joltage_idx]
        button_eq = Sum()
        for button_idx,button in enumerate(buttons):
            if joltage_idx in button:
                button_eq += opt_solver_vars[button_idx]
        opt_solver.add(button_eq == target_joltage)

    opt_solver.minimize(sum(opt_solver_vars))

    opt_solver.check()

    this_button_pushes = 0
    for button_pushes in opt_solver.model().decls():
        this_button_pushes += opt_solver.model()[button_pushes].as_long()

    #print(this_button_pushes)
    total_button_pushes += this_button_pushes

total_button_pushes




17424